In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import skimage.io
import os
from PIL import Image
from os import listdir
%matplotlib inline


C:\Users\arpit\AppData\Local\Temp\ipykernel_22344\1521909780.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from PIL import Image
import os

# Specify the path to your image folder
folder_path = 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images'


# List all files in the folder
file_list = os.listdir(folder_path)

# Load images into a list
images = []
for file in file_list:
    # Check if the file is an image
    if file.endswith('.jpg') or file.endswith('.png'):
        # Open the image and append it to the list
        img = Image.open(os.path.join(folder_path, file))
        images.append(img)

# Now 'images' contains all the images from the folder


In [3]:
base_dir_train= 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images'

In [4]:
images_train=os.listdir(base_dir_train)
type(images_train)
len(images_train)

3662

In [5]:
import os
# Get a list of all files in the directory
file_list = os.listdir(base_dir_train)

# Filter the list to include only image files (extensions: .png, .jpg, .jpeg, .gif, .bmp)
image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp']
image_files_train = [os.path.join(base_dir_train, file_name) for file_name in file_list if any(file_name.lower().endswith(ext) for ext in image_extensions)]

# Now 'image_files' is a list containing the file paths of all image files in the directory
print(image_files_train)


['C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\000c1434d8d7.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\001639a390f0.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\0024cdab0c1e.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\002c21358ce6.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\005b95c28852.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\0083ee8054ee.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\0097f532ac9f.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\00a8624548a9.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\00b74780d31d.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\00cb6555d108.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\00cc2b75cddd.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\00e4ddff966a.png', 'C:/Users/arpit/Documents/mthd=-/Green_Channel_Images\\00f6c1be5a33.png', 'C:/Users/arpit/Documents/mthd=-/Gree

In [6]:
import cv2

def histogram_equalization(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply histogram equalization
    equalized = cv2.equalizeHist(gray)
    return equalized

def apply_clahe(image, clip_limit=2.0, grid_size=(8, 8)):
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    # Apply CLAHE
    clahe_output = clahe.apply(image)
    return clahe_output

def apply_adaptive_median(image, kernel_size=3):
    # Apply Adaptive Median filter
    filtered_image = cv2.medianBlur(image, kernel_size)
    return filtered_image
    
def equalize_images_with_adaptive_median(image_paths):
    equalized_images = []
    for path in image_paths:
        # Read the image
        image = cv2.imread(path)
        # Apply histogram equalization
        equalized_image = histogram_equalization(image)
        # Apply CLAHE
        clahe_output = apply_clahe(equalized_image)
        # Apply Adaptive Median filter
        blurred_image = apply_adaptive_median(clahe_output)
        equalized_images.append(blurred_image)
    return equalized_images

# Example usage:
image_paths = image_files_train  # Assuming image_files_train contains the paths to your image files
equalized_images_with_adaptive_median = equalize_images_with_adaptive_median(image_paths)


In [7]:
# Assuming equalized_images_with_gaussian contains the processed images
# Display the first 10 images
plt.figure(figsize=(20, 10))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(cv2.cvtColor(equalized_images_with_adaptive_median[i], cv2.COLOR_BGR2RGB))
    plt.title(f'Image {i+1}')
    plt.axis('off')

plt.show()


In [9]:
import cv2
import numpy as np
import pandas as pd
import os
from skimage import feature

image_path = image_files_train
# Function to extract LBP features from an image
def extract_lbp_features(image):
    if len(image.shape) > 2:  # Check if the image is not grayscale
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    radius = 3
    n_points = 8 * radius
    method = 'uniform'
    lbp_image = feature.local_binary_pattern(image, n_points, radius, method)
    hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist

# Initialize lists to store image names and features
image_names = []
lbp_features_list = []

# Extract LBP features for each image in the list equalized_images_with_gaussian
for i, image in enumerate(equalized_images_with_adaptive_median):
    features = extract_lbp_features(image)
    lbp_features_list.append(features)

    # Store image names as 'Image_X' where X is the index of the image in the list
    image_name = os.path.splitext(os.path.basename(image_path[i]))[0]
    image_names.append(image_name)

# Convert the lists of names and features to a DataFrame
data = np.array(lbp_features_list)
df = pd.DataFrame(data, columns=[f'Feature_{i}' for i in range(data.shape[1])])
df.insert(0, 'Image_Name', image_names)

# Save the DataFrame to a CSV file
df.to_csv('lbp_features_with_names_equalized_images_clahe_median_eq.csv', index=False)


In [10]:
df=pd.read_csv('lbp_features_with_names_equalized_images_clahe_median_eq.csv')

In [11]:
df

,Image_Name,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25
0,000c1434d8d7,0.028336,0.009177,0.015157,0.018080,0.013308,0.013437,0.011295,0.012500,0.016324,...,0.024701,0.032710,0.023415,0.027404,0.028188,0.027133,0.020222,0.014472,0.254225,0.243823
1,001639a390f0,0.017334,0.005381,0.012232,0.017162,0.009867,0.009802,0.008631,0.010357,0.015234,...,0.030041,0.040977,0.026571,0.028962,0.028325,0.027800,0.020683,0.015006,0.269915,0.196693
2,0024cdab0c1e,0.040213,0.012529,0.019784,0.020234,0.015790,0.014370,0.010921,0.011263,0.012384,...,0.018123,0.026548,0.020863,0.027161,0.030883,0.030050,0.023285,0.015427,0.222226,0.309517
3,002c21358ce6,0.046318,0.012378,0.018810,0.020516,0.017371,0.016021,0.013846,0.014593,0.016691,...,0.020010,0.024184,0.019090,0.022621,0.025233,0.026479,0.020779,0.014679,0.178044,0.281779
4,005b95c28852,0.031219,0.010322,0.011769,0.011177,0.009744,0.008256,0.006824,0.006651,0.006601,...,0.007370,0.010156,0.008049,0.009932,0.012125,0.014212,0.012806,0.009637,0.561139,0.195185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,ffa47f6a7bf4,0.029721,0.009042,0.014767,0.017593,0.014127,0.014445,0.012217,0.013491,0.016421,...,0.022531,0.028641,0.022453,0.026796,0.027534,0.024897,0.018723,0.012546,0.274503,0.241328
3658,ffc04fed30e6,0.030400,0.010398,0.011412,0.010800,0.009332,0.007934,0.006643,0.006728,0.006920,...,0.007654,0.010120,0.008069,0.009833,0.011841,0.013794,0.012330,0.009635,0.559411,0.191715
3659,ffcf7b45f213,0.018328,0.005706,0.011866,0.016335,0.010111,0.010334,0.009257,0.011153,0.016314,...,0.030433,0.039558,0.025859,0.027641,0.027447,0.026630,0.019971,0.014765,0.256824,0.191196
3660,ffd97f8cd5aa,0.048668,0.013410,0.019571,0.020724,0.017965,0.016081,0.013367,0.013556,0.015726,...,0.018966,0.022818,0.018229,0.021507,0.024622,0.026641,0.021772,0.014873,0.183746,0.290600


In [12]:
df_csv = pd.read_csv('train.csv')

In [13]:
merged_df = pd.merge(df, df_csv, left_on='Image_Name', right_on='id_code', how='inner')
print(merged_df)

        Image_Name  Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  \
0     000c1434d8d7   0.028336   0.009177   0.015157   0.018080   0.013308   
1     001639a390f0   0.017334   0.005381   0.012232   0.017162   0.009867   
2     0024cdab0c1e   0.040213   0.012529   0.019784   0.020234   0.015790   
3     002c21358ce6   0.046318   0.012378   0.018810   0.020516   0.017371   
4     005b95c28852   0.031219   0.010322   0.011769   0.011177   0.009744   
...            ...        ...        ...        ...        ...        ...   
3657  ffa47f6a7bf4   0.029721   0.009042   0.014767   0.017593   0.014127   
3658  ffc04fed30e6   0.030400   0.010398   0.011412   0.010800   0.009332   
3659  ffcf7b45f213   0.018328   0.005706   0.011866   0.016335   0.010111   
3660  ffd97f8cd5aa   0.048668   0.013410   0.019571   0.020724   0.017965   
3661  ffec9a18a3ce   0.034144   0.016446   0.015175   0.014585   0.010724   

      Feature_5  Feature_6  Feature_7  Feature_8  ...  Feature_18  Feature_

In [14]:
merged_df = merged_df.drop('id_code', axis =1)

In [15]:
merged_df

,Image_Name,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,diagnosis
0,000c1434d8d7,0.028336,0.009177,0.015157,0.018080,0.013308,0.013437,0.011295,0.012500,0.016324,...,0.032710,0.023415,0.027404,0.028188,0.027133,0.020222,0.014472,0.254225,0.243823,2
1,001639a390f0,0.017334,0.005381,0.012232,0.017162,0.009867,0.009802,0.008631,0.010357,0.015234,...,0.040977,0.026571,0.028962,0.028325,0.027800,0.020683,0.015006,0.269915,0.196693,4
2,0024cdab0c1e,0.040213,0.012529,0.019784,0.020234,0.015790,0.014370,0.010921,0.011263,0.012384,...,0.026548,0.020863,0.027161,0.030883,0.030050,0.023285,0.015427,0.222226,0.309517,1
3,002c21358ce6,0.046318,0.012378,0.018810,0.020516,0.017371,0.016021,0.013846,0.014593,0.016691,...,0.024184,0.019090,0.022621,0.025233,0.026479,0.020779,0.014679,0.178044,0.281779,0
4,005b95c28852,0.031219,0.010322,0.011769,0.011177,0.009744,0.008256,0.006824,0.006651,0.006601,...,0.010156,0.008049,0.009932,0.012125,0.014212,0.012806,0.009637,0.561139,0.195185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,ffa47f6a7bf4,0.029721,0.009042,0.014767,0.017593,0.014127,0.014445,0.012217,0.013491,0.016421,...,0.028641,0.022453,0.026796,0.027534,0.024897,0.018723,0.012546,0.274503,0.241328,2
3658,ffc04fed30e6,0.030400,0.010398,0.011412,0.010800,0.009332,0.007934,0.006643,0.006728,0.006920,...,0.010120,0.008069,0.009833,0.011841,0.013794,0.012330,0.009635,0.559411,0.191715,0
3659,ffcf7b45f213,0.018328,0.005706,0.011866,0.016335,0.010111,0.010334,0.009257,0.011153,0.016314,...,0.039558,0.025859,0.027641,0.027447,0.026630,0.019971,0.014765,0.256824,0.191196,2
3660,ffd97f8cd5aa,0.048668,0.013410,0.019571,0.020724,0.017965,0.016081,0.013367,0.013556,0.015726,...,0.022818,0.018229,0.021507,0.024622,0.026641,0.021772,0.014873,0.183746,0.290600,0


In [16]:
merged_df['diagnosis'] = merged_df['diagnosis'].apply(lambda x: 1 if x != 0 else x)

In [17]:
merged_df['diagnosis']

0       1
1       1
2       1
3       0
4       0
       ..
3657    1
3658    0
3659    1
3660    0
3661    1
Name: diagnosis, Length: 3662, dtype: int64

In [18]:
merged_df

,Image_Name,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,diagnosis
0,000c1434d8d7,0.028336,0.009177,0.015157,0.018080,0.013308,0.013437,0.011295,0.012500,0.016324,...,0.032710,0.023415,0.027404,0.028188,0.027133,0.020222,0.014472,0.254225,0.243823,1
1,001639a390f0,0.017334,0.005381,0.012232,0.017162,0.009867,0.009802,0.008631,0.010357,0.015234,...,0.040977,0.026571,0.028962,0.028325,0.027800,0.020683,0.015006,0.269915,0.196693,1
2,0024cdab0c1e,0.040213,0.012529,0.019784,0.020234,0.015790,0.014370,0.010921,0.011263,0.012384,...,0.026548,0.020863,0.027161,0.030883,0.030050,0.023285,0.015427,0.222226,0.309517,1
3,002c21358ce6,0.046318,0.012378,0.018810,0.020516,0.017371,0.016021,0.013846,0.014593,0.016691,...,0.024184,0.019090,0.022621,0.025233,0.026479,0.020779,0.014679,0.178044,0.281779,0
4,005b95c28852,0.031219,0.010322,0.011769,0.011177,0.009744,0.008256,0.006824,0.006651,0.006601,...,0.010156,0.008049,0.009932,0.012125,0.014212,0.012806,0.009637,0.561139,0.195185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,ffa47f6a7bf4,0.029721,0.009042,0.014767,0.017593,0.014127,0.014445,0.012217,0.013491,0.016421,...,0.028641,0.022453,0.026796,0.027534,0.024897,0.018723,0.012546,0.274503,0.241328,1
3658,ffc04fed30e6,0.030400,0.010398,0.011412,0.010800,0.009332,0.007934,0.006643,0.006728,0.006920,...,0.010120,0.008069,0.009833,0.011841,0.013794,0.012330,0.009635,0.559411,0.191715,0
3659,ffcf7b45f213,0.018328,0.005706,0.011866,0.016335,0.010111,0.010334,0.009257,0.011153,0.016314,...,0.039558,0.025859,0.027641,0.027447,0.026630,0.019971,0.014765,0.256824,0.191196,1
3660,ffd97f8cd5aa,0.048668,0.013410,0.019571,0.020724,0.017965,0.016081,0.013367,0.013556,0.015726,...,0.022818,0.018229,0.021507,0.024622,0.026641,0.021772,0.014873,0.183746,0.290600,0


In [19]:
#from sklearn.metrics import confusion_matrix

def calculate_distances_with_names(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    distances = np.linalg.norm(database_features - query_features, ord =1,axis=1)
    image_names = image_database['Image_Name'].values
    return distances, image_names

def retrieve_similar_images(query_image, image_database):
    distances, image_names = calculate_distances_with_names(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[0:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []
all_predicted_labels = []
all_true_labels = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

        # Retrieving similar images
    similar_images_indices = retrieve_similar_images(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

        # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images /10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)


# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.9108410704533042
Average Recall: 0.004975073462397078


In [20]:
#from sklearn.metrics import confusion_matrix

def calculate_distances_with_names(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    distances = np.linalg.norm(database_features - query_features, axis=1)
    image_names = image_database['Image_Name'].values
    return distances, image_names

def retrieve_similar_images(query_image, image_database):
    distances, image_names = calculate_distances_with_names(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[0:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []
all_predicted_labels = []
all_true_labels = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

        # Retrieving similar images
    similar_images_indices = retrieve_similar_images(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

        # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images /10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)


# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.9015019115237576
Average Recall: 0.004924057392335008


In [21]:
def calculate_distances_with_names_jaccard(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    # Convert binary values based on a threshold (e.g., >0.5 as 1, else 0)
    query_features_bin = (query_features > 0.5).astype(int)
    database_features_bin = (database_features > 0.5).astype(int)

    # Compute Jaccard similarity
    intersection = np.sum(np.minimum(query_features_bin, database_features_bin), axis=1)
    union = np.sum(np.maximum(query_features_bin, database_features_bin), axis=1)

    # Avoid division by zero, replace the division by zero with NaN
    with np.errstate(divide='ignore', invalid='ignore'):  # Ignore division by zero errors
        jaccard_sim = np.divide(intersection, union, out=np.zeros_like(union, dtype=float), where=union != 0)
        distances = 1 - jaccard_sim  # Convert similarity to distance

    image_names = image_database['Image_Name'].values
    return distances, image_names



# Use calculate_distances_with_names_jaccard in your loop for Jaccard similarity

def retrieve_similar_images_jaccard(query_image, image_database):
    distances, image_names = calculate_distances_with_names_jaccard(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

    # Retrieving similar images
    similar_images_indices = retrieve_similar_images_jaccard(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

    # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images / 10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)

# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.5980338612779902
Average Recall: 0.003258138159218507


In [22]:
def calculate_distances_with_names_cosine(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9','Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    # Calculate dot product between query and database vectors
    dot_product = np.dot(database_features, query_features)

    # Calculate magnitudes of the vectors
    query_magnitude = np.linalg.norm(query_features)
    database_magnitudes = np.linalg.norm(database_features, axis=1)

    # Calculate cosine similarity
    cosine_sim = dot_product / (query_magnitude * database_magnitudes)

    # Convert similarity to distance
    distances = 1 - cosine_sim

    image_names = image_database['Image_Name'].values
    return distances, image_names

# Use calculate_distances_with_names_cosine in your loop for Cosine Similarity

def retrieve_similar_images_cosine(query_image, image_database):
    distances, image_names = calculate_distances_with_names_cosine(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

    # Retrieving similar images
    similar_images_indices = retrieve_similar_images_cosine(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

    # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images / 10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)

# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.9043965046422721
Average Recall: 0.004939949882060829


In [23]:
# Assuming 'df' is your DataFrame
rows_with_diagnosis_0 = merged_df[merged_df['diagnosis'] == 0]
rows_with_diagnosis_1 = merged_df[merged_df['diagnosis'] == 1]


In [24]:
rows_with_diagnosis_0

,Image_Name,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_17,Feature_18,Feature_19,Feature_20,Feature_21,Feature_22,Feature_23,Feature_24,Feature_25,diagnosis
3,002c21358ce6,0.046318,0.012378,0.018810,0.020516,0.017371,0.016021,0.013846,0.014593,0.016691,...,0.024184,0.019090,0.022621,0.025233,0.026479,0.020779,0.014679,0.178044,0.281779,0
4,005b95c28852,0.031219,0.010322,0.011769,0.011177,0.009744,0.008256,0.006824,0.006651,0.006601,...,0.010156,0.008049,0.009932,0.012125,0.014212,0.012806,0.009637,0.561139,0.195185,0
6,0097f532ac9f,0.034603,0.014898,0.015275,0.015817,0.013283,0.014630,0.011835,0.013373,0.015993,...,0.023231,0.019783,0.024545,0.029724,0.028618,0.027599,0.015021,0.133475,0.338115,0
10,00cc2b75cddd,0.043657,0.012177,0.018964,0.019714,0.016871,0.015632,0.013034,0.014207,0.016600,...,0.025046,0.019569,0.023348,0.026307,0.026823,0.021133,0.014270,0.165046,0.280233,0
12,00f6c1be5a33,0.047285,0.013383,0.019492,0.019759,0.016631,0.014727,0.012515,0.013471,0.015688,...,0.021695,0.017448,0.020931,0.023654,0.025822,0.021016,0.014206,0.199547,0.286278,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,ff4cd992667b,0.032108,0.013900,0.014436,0.014723,0.012578,0.013948,0.011584,0.013432,0.016311,...,0.023313,0.019686,0.024145,0.028564,0.028077,0.026383,0.015232,0.146118,0.322929,0
3653,ff59d44a70a7,0.050727,0.013285,0.020384,0.021941,0.019110,0.016617,0.013886,0.014438,0.015760,...,0.022630,0.018582,0.022509,0.025822,0.027692,0.022708,0.015418,0.164951,0.292752,0
3654,ff631653374e,0.050016,0.013723,0.019833,0.020473,0.017316,0.015022,0.012756,0.013150,0.015111,...,0.021087,0.016725,0.020124,0.023371,0.025899,0.021227,0.014348,0.198649,0.297187,0
3658,ffc04fed30e6,0.030400,0.010398,0.011412,0.010800,0.009332,0.007934,0.006643,0.006728,0.006920,...,0.010120,0.008069,0.009833,0.011841,0.013794,0.012330,0.009635,0.559411,0.191715,0


In [28]:
#from sklearn.metrics import confusion_matrix

def calculate_distances_with_names(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    distances = np.linalg.norm(database_features - query_features, ord =1,axis=1)
    image_names = image_database['Image_Name'].values
    return distances, image_names

def retrieve_similar_images(query_image, image_database):
    distances, image_names = calculate_distances_with_names(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[0:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []
all_predicted_labels = []
all_true_labels = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

        # Retrieving similar images
    similar_images_indices = retrieve_similar_images(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

        # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images /10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)


# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.8854997269251775
Average Recall: 0.004836419363521129


In [25]:
#from sklearn.metrics import confusion_matrix

def calculate_distances_with_names(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9','Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    distances = np.linalg.norm(database_features - query_features, axis=1)
    image_names = image_database['Image_Name'].values
    return distances, image_names

def retrieve_similar_images(query_image, image_database):
    distances, image_names = calculate_distances_with_names(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[0:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []
all_predicted_labels = []
all_true_labels = []

for i in range(1857):  # Loop through all query images
    query_image = rows_with_diagnosis_1.iloc[i]
    image_database = merged_df  # Your main dataset containing images

        # Retrieving similar images
    similar_images_indices = retrieve_similar_images(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

        # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images /10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)


# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.8950457727517501
Average Recall: 0.00481984799543215


In [29]:
def calculate_distances_with_names_jaccard(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    # Convert binary values based on a threshold (e.g., >0.5 as 1, else 0)
    query_features_bin = (query_features > 0.5).astype(int)
    database_features_bin = (database_features > 0.5).astype(int)

    # Compute Jaccard similarity
    intersection = np.sum(np.minimum(query_features_bin, database_features_bin), axis=1)
    union = np.sum(np.maximum(query_features_bin, database_features_bin), axis=1)

    # Avoid division by zero, replace the division by zero with NaN
    with np.errstate(divide='ignore', invalid='ignore'):  # Ignore division by zero errors
        jaccard_sim = np.divide(intersection, union, out=np.zeros_like(union, dtype=float), where=union != 0)
        distances = 1 - jaccard_sim  # Convert similarity to distance

    image_names = image_database['Image_Name'].values
    return distances, image_names



# Use calculate_distances_with_names_jaccard in your loop for Jaccard similarity

def retrieve_similar_images_jaccard(query_image, image_database):
    distances, image_names = calculate_distances_with_names_jaccard(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

    # Retrieving similar images
    similar_images_indices = retrieve_similar_images_jaccard(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

    # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images / 10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)

# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.5028399781540143
Average Recall: 0.0027307482250136543


In [30]:
def calculate_distances_with_names_cosine(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    # Calculate dot product between query and database vectors
    dot_product = np.dot(database_features, query_features)

    # Calculate magnitudes of the vectors
    query_magnitude = np.linalg.norm(query_features)
    database_magnitudes = np.linalg.norm(database_features, axis=1)

    # Calculate cosine similarity
    cosine_sim = dot_product / (query_magnitude * database_magnitudes)

    # Convert similarity to distance
    distances = 1 - cosine_sim

    image_names = image_database['Image_Name'].values
    return distances, image_names

# Use calculate_distances_with_names_cosine in your loop for Cosine Similarity

def retrieve_similar_images_cosine(query_image, image_database):
    distances, image_names = calculate_distances_with_names_cosine(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

    # Retrieving similar images
    similar_images_indices = retrieve_similar_images_cosine(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

    # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images / 10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)

# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.8713544511196067
Average Recall: 0.004759352756151532


In [26]:
#from sklearn.metrics import confusion_matrix

def calculate_distances_with_names(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    distances = np.linalg.norm(database_features - query_features, axis=1)
    image_names = image_database['Image_Name'].values
    return distances, image_names

def retrieve_similar_images(query_image, image_database):
    distances, image_names = calculate_distances_with_names(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[0:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []
all_predicted_labels = []
all_true_labels = []

for i in range(1805):  # Loop through all query images
    query_image = rows_with_diagnosis_0.iloc[i]
    image_database = merged_df  # Your main dataset containing images

        # Retrieving similar images
    similar_images_indices = retrieve_similar_images(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

        # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images /10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)


# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.9081440443213297
Average Recall: 0.005031268943608476


In [27]:
#from sklearn.metrics import confusion_matrix

def calculate_distances_with_names(query_image, image_database):
    query_features = query_image[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                  'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                  'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    database_features = image_database[['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3',
                                        'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7',
                                        'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13',
                                  'Feature_14', 'Feature_15', 'Feature_16', 'Feature_17',
                                  'Feature_18', 'Feature_19', 'Feature_20', 'Feature_21', 'Feature_22', 'Feature_23',
                                  'Feature_24', 'Feature_25']].values
    query_features = query_features.astype(float)
    database_features = database_features.astype(float)

    distances = np.linalg.norm(database_features - query_features, axis=1)
    image_names = image_database['Image_Name'].values
    return distances, image_names

def retrieve_similar_images(query_image, image_database):
    distances, image_names = calculate_distances_with_names(query_image, image_database)
    distances_with_names = list(zip(range(len(distances)), distances, image_names))
    distances_with_names = sorted(distances_with_names, key=lambda x: x[1], reverse=False)[0:10]
    sorted_indices = [i[0] for i in distances_with_names]
    similar_images_indices = sorted_indices
    return similar_images_indices

precision_list = []
recall_list = []
all_predicted_labels = []
all_true_labels = []

for i in range(3662):  # Loop through all query images
    query_image = merged_df.iloc[i]
    image_database = merged_df  # Your main dataset containing images

        # Retrieving similar images
    similar_images_indices = retrieve_similar_images(query_image, image_database)

    # Extracting classes of query image and retrieved images
    query_class = query_image['diagnosis']
    retrieved_classes = image_database.iloc[similar_images_indices]['diagnosis'].values

        # Calculating precision and recall
    relevant_images = np.count_nonzero(retrieved_classes == query_class)
    precision = relevant_images /10  # Assuming you retrieve top 10 similar images
    recall = relevant_images / len(image_database[image_database['diagnosis'] == query_class])

    precision_list.append(precision)
    recall_list.append(recall)


# Computing average precision and recall
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)


Average Precision: 0.9015019115237576
Average Recall: 0.004924057392335008
